# Imports

In [ ]:
import pandas as pd
import json
from collections import defaultdict
import time
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import os

# Load Data

In [ ]:
df = pd.read_csv(r"/root/.cache/kagglehub/datasets/zvikinozadze/myfitnesspal-dataset/versions/1/mfp-diaries.tsv", sep = "\t", header=None, nrows=1000)

In [ ]:
df.head()

,0,1,2,3
0,1,2014-09-14,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,1,2014-09-15,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,1,2014-09-16,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,1,2014-09-17,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,1,2014-09-18,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [ ]:
df.rename(columns={0: "ID", 1: "Date", 2: "Food", 3: "Total"}, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      1000 non-null   int64 
 1   Date    1000 non-null   object
 2   Food    1000 non-null   object
 3   Total   1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [ ]:
df["Food"].iloc[0]

'[{"meal": "MY food", "dishes": [{"nutritions": [{"name": "Calories", "value": "412"}, {"name": "Carbs", "value": "29"}, {"name": "Fat", "value": "24"}, {"name": "Protein", "value": "21"}, {"name": "Sodium", "value": "258"}, {"name": "Sugar", "value": "29"}], "name": "my - McDonalds Espresso Pronto\\u00ae Flat White, 2 TALL"}, {"nutritions": [{"name": "Calories", "value": "170"}, {"name": "Carbs", "value": "25"}, {"name": "Fat", "value": "5"}, {"name": "Protein", "value": "20"}, {"name": "Sodium", "value": "260"}, {"name": "Sugar", "value": "2"}], "name": "Quest Bar - Banana Nut Muffin Natural Protein Bar, 60 g"}, {"nutritions": [{"name": "Calories", "value": "176"}, {"name": "Carbs", "value": "33"}, {"name": "Fat", "value": "1"}, {"name": "Protein", "value": "5"}, {"name": "Sodium", "value": "195"}, {"name": "Sugar", "value": "0"}], "name": "Uncle Tobys Australia - Vita Brits, 3 Biscuits 33.3g"}, {"nutritions": [{"name": "Calories", "value": "342"}, {"name": "Carbs", "value": "34"}, {

In [ ]:
type(df["Food"].iloc[0])

str

In [ ]:
json.loads(df["Food"].iloc[0])

[{'meal': 'MY food',
  'dishes': [{'nutritions': [{'name': 'Calories', 'value': '412'},
     {'name': 'Carbs', 'value': '29'},
     {'name': 'Fat', 'value': '24'},
     {'name': 'Protein', 'value': '21'},
     {'name': 'Sodium', 'value': '258'},
     {'name': 'Sugar', 'value': '29'}],
    'name': 'my - McDonalds Espresso Pronto® Flat White, 2 TALL'},
   {'nutritions': [{'name': 'Calories', 'value': '170'},
     {'name': 'Carbs', 'value': '25'},
     {'name': 'Fat', 'value': '5'},
     {'name': 'Protein', 'value': '20'},
     {'name': 'Sodium', 'value': '260'},
     {'name': 'Sugar', 'value': '2'}],
    'name': 'Quest Bar - Banana Nut Muffin Natural Protein Bar, 60 g'},
   {'nutritions': [{'name': 'Calories', 'value': '176'},
     {'name': 'Carbs', 'value': '33'},
     {'name': 'Fat', 'value': '1'},
     {'name': 'Protein', 'value': '5'},
     {'name': 'Sodium', 'value': '195'},
     {'name': 'Sugar', 'value': '0'}],
    'name': 'Uncle Tobys Australia - Vita Brits, 3 Biscuits 33.3g'},
 

In [ ]:
type(json.loads(df["Food"].iloc[0]))

list

In [ ]:
keys = json.loads(df["Food"].iloc[0])[0].keys()
keys

dict_keys(['meal', 'dishes', 'sequence'])

In [ ]:
# Json Structure

data = json.loads(df["Food"].iloc[0])

# Function to recursively print the structure in a general format
def print_structure(obj, level=0):
    indent = "  " * level  # Indentation for hierarchy

    if isinstance(obj, dict):
        for key, value in obj.items():
            print(f"{indent}- {key}")  # Print key
            print_structure(value, level + 1)  # Recurse into value
    elif isinstance(obj, list):
        print(f"{indent}- [List]")  # Indicate a list
        if obj:
            print_structure(obj[0], level + 1)  # Print structure of first element

print_structure(data)

- [List]
  - meal
  - dishes
    - [List]
      - nutritions
        - [List]
          - name
          - value
      - name
  - sequence


In [ ]:
json.loads(df["Total"].iloc[500])

{'total': [{'name': 'Calories', 'value': 514},
  {'name': 'Carbs', 'value': 60},
  {'name': 'Fat', 'value': 21},
  {'name': 'Protein', 'value': 27}],
 'goal': [{'name': 'Calories', 'value': 1700},
  {'name': 'Carbs', 'value': 170},
  {'name': 'Fat', 'value': 57},
  {'name': 'Protein', 'value': 127}]}

In [ ]:
print_structure(json.loads(df["Total"].iloc[500]))

- total
  - [List]
    - name
    - value
- goal
  - [List]
    - name
    - value


# Refactor Data

In [ ]:
def safe_float(value):
    if value is None:
        return 0.0
    try:
        return float(str(value).replace(",", ""))
    except (ValueError, TypeError):
        return 0.0

def process_row(row):
    results = {
        "Meals": "", "Dishes": "", "Num_Meals": 0,
    }
    nutrition_totals = defaultdict(float)
    nutrition_data = {}

    # --- Processes Food Data ---
    try:
        food_list = json.loads(row['Food']) if isinstance(row['Food'], str) else None
        if isinstance(food_list, list):
            meals = []
            dishes = []
            for meal in food_list:
                meals.append(meal.get("meal", "Unknown Meal"))
                for dish in meal.get("dishes", []):
                    dishes.append(dish.get("name", "Unknown Dish"))
                    for nutrient in dish.get("nutritions", []):
                        name = nutrient.get("name")
                        value = safe_float(nutrient.get("value"))
                        if name:
                            nutrition_totals[name + "_calc"] += value

            results["Meals"] = ", ".join(meals)
            results["Dishes"] = ", ".join(dishes)
            results["Num_Meals"] = len(meals)
            results.update(nutrition_totals)
        elif food_list is not None:
             results["error_food"] = "food_json_not_list"
        if "error_food" not in results: results["error_food"] = None

    except (json.JSONDecodeError, TypeError, Exception) as e:
        results["error_food"] = f"food_processing_error: {type(e).__name__}"
        if "Meals" not in results: results["Meals"] = ""
        if "Dishes" not in results: results["Dishes"] = ""
        if "Num_Meals" not in results: results["Num_Meals"] = 0


    # --- Processes Total Data ---
    try:
        total_dict = json.loads(row['Total']) if isinstance(row['Total'], str) else None
        if isinstance(total_dict, dict):
            for nutrient in total_dict.get("total", []):
                name = nutrient.get("name")
                value = safe_float(nutrient.get("value"))
                if name:
                    nutrition_data[name + "_total"] = value
            for nutrient in total_dict.get("goal", []):
                name = nutrient.get("name")
                value = safe_float(nutrient.get("value"))
                if name:
                    nutrition_data[name + "_goal"] = value
            results.update(nutrition_data)
        elif total_dict is not None:
             results["error_total"] = "total_json_not_dict"
        if "error_total" not in results: results["error_total"] = None # Or ""


    except (json.JSONDecodeError, TypeError, Exception) as e:
        results["error_total"] = f"total_processing_error: {type(e).__name__}"

    return pd.Series(results)

# --- Helper Function for Default Values ---
def get_default_value(pa_type):
    """Returns pd.NA (Pandas scalar missing value) to represent null."""
    return pd.NA

In [ ]:
chunksize = 100000
input_file = r"/root/.cache/kagglehub/datasets/zvikinozadze/myfitnesspal-dataset/versions/1/mfp-diaries.tsv"
output_file = r"/content/drive/MyDrive/My_Fitness_Pal/refined_data.parquet"

In [ ]:
# --- Main Processing Loop ---
pq_writer = None
schema = None
schema_defined = False

print("Starting processing...")
chunk_num = 0
try:
    for chunk in pd.read_csv(input_file, sep="\t", header=None, chunksize=chunksize, names=["ID", "Date", "Food", "Total"]):
        chunk_num += 1
        print(f"Processing chunk {chunk_num}...")

        # Applying processing function
        processed_data = chunk.apply(process_row, axis=1)

        # Combining with essential original columns
        df_refined_chunk = pd.concat([chunk[['ID', 'Date']], processed_data], axis=1)

        # --- Schema Definition and Alignment ---
        if not schema_defined:
            print("Defining schema from first chunk...")
            try:
                pa_table = pa.Table.from_pandas(df_refined_chunk, preserve_index=False)
                schema = pa_table.schema
                # Initializes Parquet writer with the schema
                pq_writer = pq.ParquetWriter(output_file, schema, compression="snappy")
                schema_defined = True
                print(f"Schema defined with {len(schema.names)} columns. Writing first chunk...")
                pq_writer.write_table(pa_table)
            except Exception as e:
                print(f"ERROR: Could not create schema or write first chunk: {e}")
                print("Stopping processing. Check the first chunk's data.")
                break # Stops if the first chunk fails severely

        else:
            # --- Align subsequent chunks to the established schema ---
            print(f"Aligning chunk {chunk_num} to schema...")
            current_columns = df_refined_chunk.columns
            expected_columns = schema.names

            # Adds missing columns with default values based on schema types
            missing_cols = set(expected_columns) - set(current_columns)
            if missing_cols:
                print(f"  Adding missing columns: {missing_cols}")
                for col_name in missing_cols:
                    try:
                        pa_type = schema.field(col_name).type
                        default_value = get_default_value(pa_type)
                        df_refined_chunk = df_refined_chunk.assign(**{col_name: default_value})

                    except KeyError:
                        print(f"  Warning: Column '{col_name}' in schema but couldn't get type info? Skipping default assignment.")
                    except Exception as e_add:
                         print(f"  Warning: Error adding default for column '{col_name}': {e_add}")


            # Ensures columns are in the exact order defined by the schema
            # Also drops any extra columns not in the schema (if any were accidentally created)
            df_refined_chunk = df_refined_chunk[expected_columns]

            # Converts aligned DataFrame to Arrow Table using the schema
            try:
                pa_table = pa.Table.from_pandas(df_refined_chunk, schema=schema, preserve_index=False)
                print(f"Writing aligned chunk {chunk_num}...")
                pq_writer.write_table(pa_table)
            except pa.lib.ArrowInvalid as e_write:
                 print(f"ERROR writing chunk {chunk_num} even after alignment: {e_write}")
                 print("This might indicate a data type mismatch wasn't fully resolved.")
                 # Saves problematic chunk for inspection
                 df_refined_chunk.to_csv(f"error_chunk_{chunk_num}.csv", index=False)
                 break


finally:
    if pq_writer:
        pq_writer.close()
        print(f"Finished processing. Output saved to {output_file}")
    elif chunk_num == 0:
         print("No chunks were processed.")
    else:
         print("Processing finished, but Parquet writer might not have been initialized or closed properly.")

Starting processing...
Processing chunk 1...
Defining schema from first chunk...
Schema defined with 52 columns. Writing first chunk...
Processing chunk 2...
Aligning chunk 2 to schema...
Writing aligned chunk 2...
Processing chunk 3...
Aligning chunk 3 to schema...
Writing aligned chunk 3...
Processing chunk 4...
Aligning chunk 4 to schema...
Writing aligned chunk 4...
Processing chunk 5...
Aligning chunk 5 to schema...
Writing aligned chunk 5...
Processing chunk 6...
Aligning chunk 6 to schema...
Writing aligned chunk 6...
Finished processing. Output saved to /content/drive/MyDrive/My_Fitness_Pal/refined_data.parquet


In [ ]:
df_pq = pd.read_parquet("/content/drive/MyDrive/My_Fitness_Pal/refined_data.parquet")

In [ ]:
df_pq.head()

,ID,Date,Calcium_calc,Calcium_goal,Calcium_total,Calories_calc,Calories_goal,Calories_total,Carbs_calc,Carbs_goal,...,Trn Fat_goal,Trn Fat_total,Vit A_calc,Vit A_goal,Vit A_total,Vit C_calc,Vit C_goal,Vit C_total,error_food,error_total
0,1,2014-09-14,NaN,NaN,NaN,2924.0,3173.0,2924.0,340.0,396.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,1,2014-09-15,NaN,NaN,NaN,2430.0,1572.0,2430.0,96.0,196.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,1,2014-09-16,NaN,NaN,NaN,1862.0,1832.0,1862.0,158.0,229.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,1,2014-09-17,NaN,NaN,NaN,2251.0,1685.0,2251.0,187.0,210.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,1,2014-09-18,NaN,NaN,NaN,2001.0,1597.0,2001.0,113.0,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [ ]:
df_pq.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,587187.0,NaN,NaN,NaN,4946.919608,2844.724722,1.0,2513.0,4928.0,7427.0,9897.0
Date,587187,208,2015-01-20,4531,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Calcium_calc,22925.0,NaN,NaN,NaN,68.212912,93.804823,0.0,28.0,55.0,93.0,6234.0
Calcium_goal,22925.0,NaN,NaN,NaN,101.006107,7.090225,80.0,100.0,100.0,100.0,200.0
Calcium_total,22925.0,NaN,NaN,NaN,68.212912,93.804823,0.0,28.0,55.0,93.0,6234.0
Calories_calc,587187.0,NaN,NaN,NaN,1424.116826,2285.640262,-445.0,1038.0,1404.0,1767.0,1200800.0
Calories_goal,587180.0,NaN,NaN,NaN,1804.617693,528.841239,0.0,1450.0,1702.0,2041.0,26068.0
Calories_total,587161.0,NaN,NaN,NaN,1421.987281,2284.531977,0.0,1038.0,1403.0,1766.0,1200800.0
Carbs_calc,573049.0,NaN,NaN,NaN,146.083953,305.314469,-48.0,85.0,142.0,195.0,210865.0
Carbs_goal,573042.0,NaN,NaN,NaN,195.154444,293.544589,0.0,150.0,189.0,236.0,154417.0


In [ ]:
df_pq.shape

(587187, 52)

In [ ]:
df_pq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587187 entries, 0 to 587186
Data columns (total 52 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              587187 non-null  int64  
 1   Date            587187 non-null  object 
 2   Calcium_calc    22925 non-null   float64
 3   Calcium_goal    22925 non-null   float64
 4   Calcium_total   22925 non-null   float64
 5   Calories_calc   587187 non-null  float64
 6   Calories_goal   587180 non-null  float64
 7   Calories_total  587161 non-null  float64
 8   Carbs_calc      573049 non-null  float64
 9   Carbs_goal      573042 non-null  float64
 10  Carbs_total     573041 non-null  float64
 11  Chol_calc       18252 non-null   float64
 12  Chol_goal       18252 non-null   float64
 13  Chol_total      18252 non-null   float64
 14  Dishes          587187 non-null  object 
 15  Fat_calc        569335 non-null  float64
 16  Fat_goal        569328 non-null  float64
 17  Fat_total 

In [ ]:
df_pq.isnull().sum()

,0
ID,0
Date,0
Calcium_calc,564262
Calcium_goal,564262
Calcium_total,564262
Calories_calc,0
Calories_goal,7
Calories_total,26
Carbs_calc,14138
Carbs_goal,14145


In [ ]:
df_pq.sample(5).T

,396558,502669,74758,109428,388008
ID,6683,8450,1291,1877,6525
Date,2014-11-25,2015-01-12,2014-11-09,2015-01-30,2015-01-27
Calcium_calc,NaN,NaN,NaN,NaN,45.0
Calcium_goal,NaN,NaN,NaN,NaN,100.0
Calcium_total,NaN,NaN,NaN,NaN,45.0
Calories_calc,1427.0,1255.0,1879.0,1540.0,419.0
Calories_goal,1400.0,1530.0,1965.0,3570.0,1480.0
Calories_total,1427.0,1255.0,1879.0,1540.0,419.0
Carbs_calc,119.0,39.0,232.0,201.0,71.0
Carbs_goal,120.0,191.0,197.0,446.0,185.0


In [ ]:
df_pq.Dishes.sample(60)

,Dishes
342604,"Austin Zoo - Animal Crackers, 1 container - 2o..."
334071,"Breakstones - Cottage Cheese Singles 2%, 4 oz,..."
347356,"Hardee's - 2/3 lb Monster Thickburger No Bun, ..."
291411,Torchys Tacos - Ranch Hand Taco (From Website)...
157637,International Delight - French Vanilla Creamer...
8663,"Liberté - Greek Honey Yoghurt, 100 g, Hairy Bi..."
330475,"Heb - Fat Free Cottage Cheese, 1/2 Cup, Laura ..."
570591,"New York Bagel Co - Wholemeal Bagels*, 90 g ba..."
530332,"Coffee - Instant, regular, 1 fl oz, Zymil (Pau..."
328844,"Lonsdale - Pro Bar 30 - Chocolate Mint, 1 bar,..."


In [ ]:
df_pq['Date'] = pd.to_datetime(df_pq['Date'], format='%Y-%m-%d')

In [ ]:
df_pq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587187 entries, 0 to 587186
Data columns (total 52 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ID              587187 non-null  int64         
 1   Date            587187 non-null  datetime64[ns]
 2   Calcium_calc    22925 non-null   float64       
 3   Calcium_goal    22925 non-null   float64       
 4   Calcium_total   22925 non-null   float64       
 5   Calories_calc   587187 non-null  float64       
 6   Calories_goal   587180 non-null  float64       
 7   Calories_total  587161 non-null  float64       
 8   Carbs_calc      573049 non-null  float64       
 9   Carbs_goal      573042 non-null  float64       
 10  Carbs_total     573041 non-null  float64       
 11  Chol_calc       18252 non-null   float64       
 12  Chol_goal       18252 non-null   float64       
 13  Chol_total      18252 non-null   float64       
 14  Dishes          587187 non-null  obj

In [ ]:
df_pq = df_pq.drop(columns=[col for col in df_pq.columns if col.endswith('_calc')])
df_pq.head()

,ID,Date,Calcium_goal,Calcium_total,Calories_goal,Calories_total,Carbs_goal,Carbs_total,Chol_goal,Chol_total,...,Sugar_goal,Sugar_total,Trn Fat_goal,Trn Fat_total,Vit A_goal,Vit A_total,Vit C_goal,Vit C_total,error_food,error_total
0,1,2014-09-14,NaN,NaN,3173.0,2924.0,396.0,340.0,NaN,NaN,...,119.0,109.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,1,2014-09-15,NaN,NaN,1572.0,2430.0,196.0,96.0,NaN,NaN,...,59.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,1,2014-09-16,NaN,NaN,1832.0,1862.0,229.0,158.0,NaN,NaN,...,69.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,1,2014-09-17,NaN,NaN,1685.0,2251.0,210.0,187.0,NaN,NaN,...,63.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,1,2014-09-18,NaN,NaN,1597.0,2001.0,199.0,113.0,NaN,NaN,...,60.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [ ]:
df_pq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587187 entries, 0 to 587186
Data columns (total 37 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ID              587187 non-null  int64         
 1   Date            587187 non-null  datetime64[ns]
 2   Calcium_goal    22925 non-null   float64       
 3   Calcium_total   22925 non-null   float64       
 4   Calories_goal   587180 non-null  float64       
 5   Calories_total  587161 non-null  float64       
 6   Carbs_goal      573042 non-null  float64       
 7   Carbs_total     573041 non-null  float64       
 8   Chol_goal       18252 non-null   float64       
 9   Chol_total      18252 non-null   float64       
 10  Dishes          587187 non-null  object        
 11  Fat_goal        569328 non-null  float64       
 12  Fat_total       569321 non-null  float64       
 13  Fiber_goal      242840 non-null  float64       
 14  Fiber_total     242838 non-null  flo

In [ ]:
df_pq.describe()

,ID,Date,Calcium_goal,Calcium_total,Calories_goal,Calories_total,Carbs_goal,Carbs_total,Chol_goal,Chol_total,...,Sodium_goal,Sodium_total,Sugar_goal,Sugar_total,Trn Fat_goal,Trn Fat_total,Vit A_goal,Vit A_total,Vit C_goal,Vit C_total
count,587187.000000,587187,22925.000000,22925.000000,587180.000000,5.871610e+05,573042.000000,573041.000000,18252.000000,18252.000000,...,390528.000000,390527.000000,368915.000000,368917.000000,1828.0,1828.000000,774.0,774.000000,2486.000000,2486.000000
mean,4946.919608,2015-01-02 01:00:17.197247232,101.006107,68.212912,1804.617693,1.421987e+03,195.154444,146.085394,302.588922,208.721893,...,2321.499424,1945.468989,64.569427,50.384593,0.0,0.516411,100.0,159.958656,121.399839,155.261062
min,1.000000,2014-09-14 00:00:00,80.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,100.0,0.000000,100.000000,0.000000
25%,2513.000000,2014-11-17 00:00:00,100.000000,28.000000,1450.000000,1.038000e+03,150.000000,85.000000,300.000000,48.000000,...,2300.000000,1018.000000,49.000000,22.000000,0.0,0.000000,100.0,31.250000,100.000000,11.000000
50%,4928.000000,2015-01-10 00:00:00,100.000000,55.000000,1702.000000,1.403000e+03,189.000000,142.000000,300.000000,131.000000,...,2300.000000,1794.000000,61.000000,43.000000,0.0,0.000000,100.0,95.000000,100.000000,74.500000
75%,7427.000000,2015-02-15 00:00:00,100.000000,93.000000,2041.000000,1.766000e+03,236.000000,195.000000,300.000000,281.000000,...,2300.000000,2638.000000,75.000000,69.000000,0.0,0.000000,100.0,183.500000,100.000000,194.750000
max,9897.000000,2015-04-09 00:00:00,200.000000,6234.000000,26068.000000,1.200800e+06,154417.000000,210865.000000,1200.000000,30378.000000,...,23009.000000,960000.000000,35055.000000,168015.000000,0.0,28.000000,100.0,2401.000000,500.000000,10622.000000
std,2844.724722,NaN,7.090225,93.804823,528.841239,2.284532e+03,293.544589,305.315663,71.057264,327.310953,...,414.463536,2316.717979,87.825467,280.191896,0.0,1.967512,0.0,223.746665,90.029122,404.642161


# Saving Refactored Data

In [ ]:
df_pq.drop(columns=['error_food','error_total'], inplace=True)

In [ ]:
df_pq.to_parquet("mfp_cleaned.parquet", engine="pyarrow", index=False)